# Searching Unstructured and Structured Data #
## Assignment 1: Retrieval models [100 points] ##
**TA**: Nikos Voskarides (n.voskarides@uva.nl)

In this assignment you will get familiar with basic information retrieval concepts. You will implement and evaluate different information retrieval ranking models and evaluate their performance.

We provide you with a Indri index. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-three assignment**, the deadline is **Monday, 22/1, at 23:59**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](https://piazza.com/class/ixoz63p156g1ts).

### Technicalities (must-read!) ###

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.

You will need the following software packages (tested with Python 3.5 inside [Anaconda](https://conda.io/docs/user-guide/install/index.html)):
   * Python 3.5 and Jupyter
   * Indri + Pyndri (Follow the installation instructions [here](https://github.com/nickvosk/pyndri/blob/master/README.md))
   * gensim [link](https://radimrehurek.com/gensim/install.html)
   * TREC Eval [link](https://github.com/usnistgov/trec_eval)

### TREC Eval primer ###
The TREC Eval utility can be downloaded and compiled as follows:

    git clone https://github.com/usnistgov/trec_eval.git
    cd trec_eval
    make

TREC Eval computes evaluation scores given two files: ground-truth information regarding relevant documents, named *query relevance* or *qrel*, and a ranking of documents for a set of queries, referred to as a *run*. The *qrel* will be supplied by us and should not be changed. For every retrieval model (or combinations thereof) you will generate a run of the top-1000 documents for every query. The format of the *run* file is as follows:

    $query_identifier Q0 $document_identifier $rank_of_document_for_query $query_document_similarity $run_identifier
    
where
   * `$query_identifier` is the unique identifier corresponding to a query (usually this follows a sequential numbering).
   * `Q0` is a legacy field that you can ignore.
   * `$document_identifier` corresponds to the unique identifier of a document (e.g., APXXXXXXX where AP denotes the collection and the Xs correspond to a unique numerical identifier).
   * `$rank_of_document_for_query` denotes the rank of the document for the particular query. This field is ignored by TREC Eval and is only maintained for legacy support. The ranks are computed by TREC Eval itself using the `$query_document_similarity` field (see next). However, it remains good practice to correctly compute this field.
   * `$query_document_similarity` is a score indicating the similarity between query and document where a higher score denotes greater similarity.
   * `$run_identifier` is an identifier of the run. This field is for your own convenience and has no purpose beyond bookkeeping.
   
For example, say we have two queries: `Q1` and `Q2` and we rank three documents (`DOC1`, `DOC2`, `DOC3`). For query `Q1`, we find the following similarity scores `score(Q1, DOC1) = 1.0`, `score(Q1, DOC2) = 0.5`, `score(Q1, DOC3) = 0.75`; and for `Q2`: `score(Q2, DOC1) = -0.1`, `score(Q2, DOC2) = 1.25`, `score(Q1, DOC3) = 0.0`. We can generate run using the following snippet:

In [134]:
import logging
import sys
import os
import numpy as np
def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))
            
            
# The following writes the run to standard output.
# In your code, you should write the runs to local
# storage in order to pass them to trec_eval.
write_run(
    model_name='example',
    data={
        'Q1': ((1.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=sys.stdout,
    max_objects_per_query=1000)

Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example
Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example


Now, imagine that we know that `DOC1` is relevant and `DOC3` is non-relevant for `Q1`. In addition, for `Q2` we only know of the relevance of `DOC3`. The query relevance file looks like:

    Q1 0 DOC1 1
    Q1 0 DOC3 0
    Q2 0 DOC3 1
    
We store the run and qrel in files `example.run` and `example.qrel` respectively on disk. We can now use TREC Eval to compute evaluation measures. In this example, we're only interested in Mean Average Precision and we'll only show this below for brevity. However, TREC Eval outputs much more information such as NDCG, recall, precision, etc.

    $ trec_eval -m all_trec -q example.qrel example.run | grep -E "^map\s"
    > map                   	Q1	1.0000
    > map                   	Q2	0.5000
    > map                   	all	0.7500
    
Now that we've discussed the output format of rankings and how you can compute evaluation measures from these rankings, we'll now proceed with an overview of the indexing framework you'll use.

### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [135]:
import pyndri

index = pyndri.Index('index/')

The loaded index can be used to access a collection of documents in an easy manner. We'll give you some examples to get some idea of what it can do, it is up to you to figure out how to use it for the remainder of the assignment.

First let's look at the number of documents, since Pyndri indexes the documents using incremental identifiers we can simply take the lowest index and the maximum document and consider the difference:

In [136]:
print("There are %d documents in this collection." % (index.maximum_document() - index.document_base()))

There are 164597 documents in this collection.


Let's take the first document out of the collection and take a look at it:

In [137]:
example_document = index.document(index.document_base())
print(example_document)

('AP890425-0001', (1360, 192, 363, 0, 880, 0, 200, 0, 894, 412, 92160, 3, 192, 0, 363, 34, 1441, 0, 174134, 0, 200, 0, 894, 412, 2652, 0, 810, 107, 49, 4903, 420, 0, 1, 48, 35, 489, 0, 35, 687, 192, 243, 0, 249311, 1877, 0, 1651, 1174, 0, 2701, 117, 412, 0, 810, 391, 245233, 1225, 5838, 16, 0, 233156, 3496, 0, 393, 17, 0, 2435, 4819, 930, 0, 0, 200, 0, 894, 0, 22, 398, 145, 0, 3, 271, 115, 0, 1176, 2777, 292, 0, 725, 192, 0, 0, 50046, 0, 1901, 1130, 0, 192, 0, 408, 0, 243779, 0, 0, 553, 192, 0, 363, 0, 3747, 0, 0, 0, 0, 1176, 0, 1239, 0, 0, 1115, 17, 0, 0, 585, 192, 1963, 0, 0, 412, 54356, 0, 773, 0, 0, 0, 192, 0, 0, 1130, 0, 363, 0, 545, 192, 0, 1174, 1901, 1130, 0, 4, 398, 145, 39, 0, 577, 0, 355, 0, 491, 0, 6025, 0, 0, 193156, 88, 34, 437, 0, 0, 1852, 0, 828, 0, 1588, 0, 0, 0, 2615, 0, 0, 107, 49, 420, 0, 0, 190, 7, 714, 2701, 0, 237, 192, 157, 0, 412, 34, 437, 0, 0, 200, 6025, 26, 0, 0, 0, 0, 363, 0, 22, 398, 145, 0, 200, 638, 126222, 6018, 0, 880, 0, 0, 161, 0, 0, 319, 894, 2701, 

Here we see a document consists of two things, a string representing the external document identifier and an integer list representing the identifiers of words that make up the document. Pyndri uses integer representations for words or terms, thus a token_id is an integer that represents a word whereas the token is the actual text of the word/term. Every id has a unique token and vice versa with the exception of stop words: words so common that there are uninformative, all of these receive the zero id.

To see what some ids and their matching tokens we take a look at the dictionary of the index:

In [138]:
token2id, id2token, _ = index.get_dictionary()
print(list(id2token.items())[:15])

[(1, 'new'), (2, 'percent'), (3, 'two'), (4, '1'), (5, 'people'), (6, 'million'), (7, '000'), (8, 'government'), (9, 'president'), (10, 'years'), (11, 'state'), (12, '2'), (13, 'states'), (14, 'three'), (15, 'time')]


Using this dictionary we can see the tokens for the (non-stop) words in our example document:

In [139]:
print([id2token[word_id] for word_id in example_document[1] if word_id > 0])

['52', 'students', 'arrested', 'takeover', 'university', 'massachusetts', 'building', 'fifty', 'two', 'students', 'arrested', 'tuesday', 'evening', 'occupying', 'university', 'massachusetts', 'building', 'overnight', 'protest', 'defense', 'department', 'funded', 'research', 'new', 'york', 'city', 'thousands', 'city', 'college', 'students', 'got', 'unscheduled', 'holiday', 'demonstrators', 'occupied', 'campus', 'administration', 'building', 'protest', 'possible', 'tuition', 'increases', 'prompting', 'officials', 'suspend', 'classes', '60', 'police', 'riot', 'gear', 'arrived', 'university', 'massachusetts', '5', 'p', 'm', 'two', 'hours', 'later', 'bus', 'drove', 'away', '29', 'students', 'camped', 'memorial', 'hall', 'students', 'charged', 'trespassing', '23', 'students', 'arrested', 'lying', 'bus', 'prevent', 'leaving', 'police', '300', 'students', 'stood', 'building', 'chanting', 'looking', 'students', 'hall', 'arrested', '35', 'students', 'occupied', 'memorial', 'hall', '1', 'p', 'm',

The reverse can also be done, say we want to look for news about the "University of Massachusetts", the tokens of that query can be converted to ids using the reverse dictionary:

In [140]:
query_tokens = index.tokenize("University of Massachusetts")
print("Query by tokens:", query_tokens)
query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
print("Query by ids with stopwords:", query_id_tokens)
query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
print("Query by ids without stopwords:", query_id_tokens)

Query by tokens: ['university', '', 'massachusetts']
Query by ids with stopwords: [200, 0, 894]
Query by ids without stopwords: [200, 894]


Naturally we can now match the document and query in the id space, let's see how often a word from the query occurs in our example document:

In [141]:
matching_words = sum([True for word_id in example_document[1] if word_id in query_id_tokens])
print("Document %s has %d word matches with query: \"%s\"." % (example_document[0], matching_words, ' '.join(query_tokens)))
print("Document %s and query \"%s\" have a %.01f%% overlap." % (example_document[0], ' '.join(query_tokens),matching_words/float(len(example_document[1]))*100))

Document AP890425-0001 has 13 word matches with query: "university  massachusetts".
Document AP890425-0001 and query "university  massachusetts" have a 2.5% overlap.


While this is certainly not everything Pyndri can do, it should give you an idea of how to use it. Please take a look at the [examples](https://github.com/cvangysel/pyndri) as it will help you a lot with this assignment.

**CAUTION**: Avoid printing out the whole index in this Notebook as it will generate a lot of output and is likely to corrupt the Notebook.

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [142]:
import collections
import io
import logging
import sys

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics

with open('./ap_88_89/topics_title', 'r') as f_topics:
    print(parse_topics([f_topics]))

OrderedDict([('51', 'Airbus Subsidies'), ('52', 'South African Sanctions'), ('53', 'Leveraged Buyouts'), ('54', 'Satellite Launch Contracts'), ('55', 'Insider Trading'), ('56', 'Prime (Lending) Rate Moves, Predictions'), ('57', 'MCI'), ('58', 'Rail Strikes'), ('59', 'Weather Related Fatalities'), ('60', 'Merit-Pay vs. Seniority'), ('61', 'Israeli Role in Iran-Contra Affair'), ('62', "Military Coups D'etat"), ('63', 'Machine Translation'), ('64', 'Hostage-Taking'), ('65', 'Information Retrieval Systems'), ('66', 'Natural Language Processing'), ('67', 'Politically Motivated Civil Disturbances'), ('68', 'Health Hazards from Fine-Diameter Fibers'), ('69', 'Attempts to Revive the SALT II Treaty'), ('70', 'Surrogate Motherhood'), ('71', 'Border Incursions'), ('72', 'Demographic Shifts in the U.S.'), ('73', 'Demographic Shifts across National Boundaries'), ('74', 'Conflicting Policy'), ('75', 'Automation'), ('76', 'U.S. Constitution - Original Intent'), ('77', 'Poaching'), ('78', 'Greenpeace'

### Task 1: Implement and compare lexical IR methods [35 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html) and 
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, 1500]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - [Positional Language Models](http://sifaka.cs.uiuc.edu/~ylv2/pub/sigir09-plm.pdf) define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). **[10 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[5 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences. This is *very important* in order to understand who the different retrieval functions behave.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

**IMPORTANT**: You should structure your code around the helper functions we provide below.

In [143]:
import time

with open('./ap_88_89/topics_title', 'r') as f_topics:
    queries = parse_topics([f_topics])

index = pyndri.Index('index/')
start_time = time.time()
num_documents = index.maximum_document() - index.document_base()

dictionary = pyndri.extract_dictionary(index)

tokenized_queries = {
    query_id: [dictionary.translate_token(token)
               for token in index.tokenize(query_string)
               if dictionary.has_token(token)]
    for query_id, query_string in queries.items()}

#TODO: Explain why we need it
query_term_counts = {
    query_id: collections.Counter(query_terms) 
    for query_id, query_terms in tokenized_queries.items()}

query_term_ids = set(
    query_term_id
    for query_term_ids in tokenized_queries.values()
    for query_term_id in query_term_ids)

print('Gathering statistics about', len(query_term_ids), 'terms.')

# inverted index creation.

document_lengths = {}
unique_terms_per_document = {}

inverted_index = collections.defaultdict(dict)
inverted_index_positions = collections.defaultdict(dict)
collection_frequencies = collections.defaultdict(int)

total_terms = 0

document_ids = []
for int_doc_id in range(index.document_base(), index.maximum_document()):
    # CHECK IF WE CAN DO THAT
    if len(index.document(int_doc_id)[1]) > 0:
        document_ids.append(int_doc_id)
        ext_doc_id, doc_token_ids = index.document(int_doc_id)
        doc_token_ids = [doc_token_id for doc_token_id in doc_token_ids if doc_token_id > 0]
        
        document_bow = collections.Counter(doc_token_ids)

        token_positions = collections.defaultdict(list)

        # Check with the TAs if we can change that
        [token_positions[token_id].append(i) for i, token_id in enumerate(doc_token_ids)]

        document_length = sum(document_bow.values())

        document_lengths[int_doc_id] = document_length
        total_terms += document_length

        unique_terms_per_document[int_doc_id] = len(document_bow)

        for query_term_id in query_term_ids:
            assert query_term_id is not None

            document_term_frequency = document_bow.get(query_term_id, 0)

            if document_term_frequency == 0:
                continue

            collection_frequencies[query_term_id] += document_term_frequency
            inverted_index[query_term_id][int_doc_id] = document_term_frequency
            inverted_index_positions[query_term_id][int_doc_id] = token_positions[query_term_id]

inverted_index = dict(inverted_index)
inverted_index_positions = dict(inverted_index_positions)

avg_doc_length = total_terms / num_documents

print('Inverted index creation took', time.time() - start_time, 'seconds.')

Gathering statistics about 456 terms.
Inverted index creation took 108.67771911621094 seconds.


In [144]:
import sys
def run_retrieval(model_name, score_fn, top_1000=None):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = '{}.run'.format(model_name)

    if os.path.exists(run_out_path):
        return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)
    
    data = collections.defaultdict(list)
    # The dictionary data should have the form: query_id --> (document_score, external_doc_id)
    count = 0
    for query_id in tokenized_queries:
            
        if count % 15 == 0:
            print('Finished {}%...'.format(count / 15 * 10))
        count += 1    
        doc_result = []
        global document_ids
        if top_1000 is not None:
            print('using top 1000')
            document_ids = top_1000
        for int_doc_id in document_ids:
            ext_doc_id, _ = index.document(int_doc_id)
            doc_score = score_fn(int_doc_id, query_id)
#             if doc_score != 0:
            data[query_id].append((doc_score, ext_doc_id))
        
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)

In [ ]:
import numpy as np
def tfidf(int_document_id, query_id):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query id
    """
    
    score = 0
    
    for query_term_id in tokenized_queries[query_id]:
        if int_document_id in inverted_index[query_term_id]:
            tf = inverted_index[query_term_id][int_document_id]
            df = len(inverted_index[query_term_id])
            idf = num_documents/df
            score += np.log(1 + tf) * np.log(idf)
            
    return score

run_retrieval('tfidf', tfidf)

In [145]:
from collections import defaultdict

ext2int = {index.ext_document_id(i): i for i in range(index.document_base(), index.maximum_document())}

def get_first_1000(measure):
    f = open('{}.run'.format(measure), 'r')
    top = defaultdict(list)
    top_values = defaultdict(list)
    
    for line in f.readlines():
        splitted = line.split(' ')
        query_id, ext_id, rank, tfidf = int(splitted[0]), splitted[2], int(splitted[3]), float(splitted[4])
        top[query_id].append(ext2int[ext_id])
        top_values[query_id].append(tfidf)
        
    f.close()
    return top, top_values

top_1000, _ = get_first_1000('tfidf')

In [ ]:
def bm25(int_document_id, query_id):
    k1=1.2
    b=0.75
    score = 0
    
    for query_term_id in set(tokenized_queries[query_id]):
        if int_document_id in inverted_index[query_term_id]:
            tf_td = inverted_index[query_term_id][int_document_id]
          
            Ld = document_lengths[int_document_id]
            Lavg = avg_doc_length

            tf_td = inverted_index[query_term_id][int_document_id]

            df = len(inverted_index[query_term_id])
            idf = num_documents/df
            tf_tq = query_term_counts[query_id][query_term_id]

            first_term = np.log(idf)
            second_term = ((k1+1)*tf_td)/(k1*((1-b)+b*(Ld/avg_doc_length))+tf_td)

            score += first_term * second_term # * third_term

    return score

run_retrieval('bm25', bm25)

In [ ]:
import functools
def jelinek_mercer(int_document_id, query_id, lambd):
    
    score = 0
    for query_term_id in tokenized_queries[query_id]:

        tf = inverted_index[query_term_id].get(int_document_id, 0)
        doc_len = document_lengths[int_document_id] 

        first_term = lambd*tf/doc_len
        second_term = (1 - lambd) * collection_frequencies[query_term_id]/total_terms
        score += np.log(first_term + second_term) * query_term_counts[query_id][query_term_id]
    
    return score

for lambd_val in [0.1, 0.5, 0.9]:
    jel_merc_func = functools.partial(jelinek_mercer, lambd = lambd_val)
    run_retrieval('jelinek_mercer_{}'.format(lambd_val), jel_merc_func)

In [ ]:
def dirichlet_prior(int_document_id, query_id, mu):
    score = 0
    for query_term_id in tokenized_queries[query_id]:
        tf = inverted_index[query_term_id].get(int_document_id, 0)
                    
        prob = collection_frequencies[query_term_id]/total_terms
        doc_len = document_lengths[int_document_id] 

        score += (np.log(tf+mu*prob)-np.log(doc_len + mu))*query_term_counts[query_id][query_term_id]
    
    return score

for mu_val in [500, 1000, 1500]:
    dir_prior_func = functools.partial(dirichlet_prior, mu=mu_val)
    run_retrieval('dirichlet_prior_{}'.format(mu_val), dir_prior_func)

In [ ]:
def abs_discount(int_document_id, query_id, delta):
    score = 0
    for query_term_id in tokenized_queries[query_id]:

        tf = inverted_index[query_term_id].get(int_document_id, 0)

        prob = collection_frequencies[query_term_id]/total_terms
        doc_len = document_lengths[int_document_id] 
        doc_unique_len = unique_terms_per_document[int_document_id]

        first_term = max(tf - delta, 0)
        second_term = delta*doc_unique_len*prob/doc_len

        score += np.log(first_term + second_term)*query_term_counts[query_id][query_term_id]
    
    return score

for delta_val in [0.1, 0.5, 0.9]:
    abs_discount_func = functools.partial(abs_discount, delta=delta_val)
    run_retrieval('abs_discounting_{}'.format(delta_val), abs_discount_func)

In [ ]:
import sys
def run_retrieval_plm(model_name, score_fn, top_1000):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = 'results/{}.run'.format(model_name)

    if os.path.exists(run_out_path):
        return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)
    
    data = collections.defaultdict(list)
    # The dictionary data should have the form: query_id --> (document_score, external_doc_id)
    count = 0
    for query_id in tokenized_queries:
            
        if count % 15 == 0:
            print('Finished {}%...'.format(count / 15 * 10))
        count += 1    
        doc_result = []
        for int_doc_id in top_1000[int(query_id)]:
            ext_doc_id, _ = index.document(int_doc_id)
            doc_score = score_fn(int_doc_id, query_id)
            data[query_id].append((doc_score, ext_doc_id))
        
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)
    

In [19]:
from scipy.stats import norm

def gauss_kernel(i, sigma=50):
    kernel_val = np.exp(-np.power(i, 2)/(2*np.power(sigma,2)))
    return kernel_val

def compute_kernel_table(kernel, integral, limit=2500):
    kernel_table = np.zeros(limit)
    integral_table = np.zeros(limit)
    for i in range(limit):
        kernel_table[i] = kernel(i)
        integral_table[i] = integral(i)
    return kernel_table, integral_table

def gauss_partial_integral(i, sigma=50):
    return np.sqrt(2*np.pi*np.power(sigma,2)) * (norm.cdf(i/sigma) - norm.cdf(0))

def triangle_kernel(i, sigma=50):
    kernel_val = 1-np.abs(i)/sigma if np.abs(i)<=sigma else 0
    return kernel_val

def triangle_partial_integral(i, sigma=50):
    return  i - np.power(i, 2)/(2*sigma) if i < sigma else sigma/2

def cosine_kernel(i, sigma=50):
    kernel_val = 1/2 * (1+np.cos(np.abs(i)*np.pi/sigma))\
        if np.abs(i)<=sigma\
        else 0
    return kernel_val

def cosine_partial_integral(i, sigma=50):
    return sigma/2 + sigma*np.sin(np.pi*i/sigma)/(2*np.pi) if i < sigma else sigma/2

def circle_kernel(i, sigma=50):
    kernel_val = np.sqrt(1-np.power(np.abs(i)/sigma,2))\
        if np.abs(i)<=sigma\
        else 0
    return kernel_val

def circle_partial_integral(i, sigma=50):
    return sigma/2 * (np.arcsin(i/sigma) + 1/2 * np.sin(2*np.arcsin(i/sigma)))\
                        if i < sigma else sigma*np.pi/4

def passage_kernel(i, sigma=50):
    kernel_val = 1 if np.abs(i)<=sigma else 0
    return kernel_val

def passage_partial_integral(i, sigma=50):
    return i if i < sigma else sigma

gauss_kernel_table, gauss_integral_table = compute_kernel_table(gauss_kernel, gauss_partial_integral)
triangle_kernel_table, triangle_integral_table = compute_kernel_table(triangle_kernel, triangle_partial_integral)
cosine_kernel_table, cosine_integral_table = compute_kernel_table(cosine_kernel, cosine_partial_integral)
circle_kernel_table, circle_integral_table = compute_kernel_table(circle_kernel, circle_partial_integral)
passage_kernel_table, passage_integral_table = compute_kernel_table(passage_kernel, passage_partial_integral)

In [ ]:
import functools
import numpy as np
import time

def positional_language_model(int_document_id, query_id, kernel=gauss_kernel_table, kernel_integral=gauss_integral_table, mu=0.1):
    all_scores = np.zeros(document_lengths[int_document_id])
    for query_term_id in set(tokenized_queries[query_id]):    
        for i in range(document_lengths[int_document_id]):
            term_score = 0
            score = 0
            if int_document_id in inverted_index_positions[query_term_id]:
                for query_term_pos in inverted_index_positions[query_term_id][int_document_id]:
                    term_score += kernel[np.abs(i-query_term_pos)]
               
                term_score = min(term_score, 1.0)
            else:
                term_score = 0
                
            prob_query = query_term_counts[query_id][query_term_id]/len(tokenized_queries[query_id])
            prob = collection_frequencies[query_term_id]/total_terms
            doc_len = document_lengths[int_document_id] 
            norm_value = kernel_integral[i] + kernel_integral[document_lengths[int_document_id]-1-i]

            prob_word = (term_score+mu*prob)/(norm_value + mu)
            score = prob_query * np.log(prob_query/prob_word)
            all_scores[i] += score
            
    all_scores *= -1    
    score = np.max(all_scores)
    return score

names = ['gauss', 'triangle', 'circle', 'cosine', 'passage']
for mu_val in [500, 1000, 1500]:

    for i, (kernel_func_table, kernel_integral_table) in enumerate([(gauss_kernel_table, gauss_integral_table), 
                                         (triangle_kernel_table, triangle_integral_table),
                                         (circle_kernel_table, circle_integral_table),
                                         (cosine_kernel_table, cosine_integral_table),
                                         (passage_kernel_table, passage_integral_table)]):
        start = time.time()
        plm_func = functools.partial(positional_language_model, kernel=kernel_func_table, kernel_integral=kernel_integral_table, mu=mu_val)
        run_retrieval_plm('plm-' + names[i] + '_' + str(mu_val), plm_func, top_1000=top_1000)
        print(time.time() - start)

In [ ]:
from collections import defaultdict
import os
import scipy.stats
import matplotlib.pyplot as plt

def validation_best(score_name, param_list, measures, valid=True):
    result_dicts = []
    for p in param_list:
        
        result_dicts.append(trec_eval(score_name + ('_' + str(p) if p != None else ''), measures, validation=valid))
    vals = []
    for res in result_dicts:
        vals.append(res['ndcg_cut_10']['all'])
    best = np.argmax(vals)
    return best, vals[best], param_list[best], result_dicts[best], result_dicts
    
def trec_eval(score_name, measures, validation=True):
    validation_str = 'validation' if validation else 'test'
    results = defaultdict(dict)
    print(score_name)
    print(os.path.curdir)
    if os.path.isfile('results/{}'.format(score_name)):
        print('weeeeeeee234342')
    else:
        print('results/{}.run'.format(score_name))
        return 5
    a = os.system(str.format('./trec_eval/trec_eval -m all_trec -q ap_88_89/qrel_{} results/{}.run > results/trec_results_{}', validation_str, score_name, score_name))
    f = open('results/trec_results_' + score_name, 'r')
    for line in f.readlines():
        print(line)
        name, id, val = line.strip().split()
        if name in measures:
            val = float(val)
            results[name][id] = val

    f.close()
    return results

In [112]:
kernel_names = ['gauss', 'triangle', 'circle', 'cosine', 'passage']
measures = ['ndcg_cut_10', 'map_cut_1000', 'recall_1000', 'P_5']
scores = ['jelinek_mercer','abs_discounting', 'dirichlet_prior', 'tfidf', 'bm25']
plm_names = ['plm-' + kernel_name for kernel_name in kernel_names]

scores = scores + plm_names
params = {'jelinek_mercer': [0.1, 0.5, 0.9],
         'abs_discounting': [0.1, 0.5, 0.9],
         'dirichlet_prior': [500, 1000, 1500],
          'tfidf': [None], 'bm25': [None]}
for i, name in enumerate(plm_names):
    params[name] = [500, 1000, 1500]
    
ind = np.arange(len(measures))
width = 0.08
colors = ['r', 'g', 'b', 'y', 'k', 'grey', 'purple', 'orange', 'pink']
colors = colors + colors + colors + colors
fig, ax = plt.subplots()
rs = []

bests = dict()
for i, score in enumerate(scores):
    print(score, params[score])
    best, best_val, best_param, best_result, results = validation_best(score, params[score], measures)
    bests[score] = best_param
    means = [best_result[measure]['all'] for measure in measures]
    rs.append(ax.bar(ind + width*i, means, width, color=colors[i]))

ax.legend(rs, scores)
ax.set_ylabel('Scores')
ax.set_title('Scores measure')
ax.set_xticks(ind + width*10 / 2)
ax.set_xticklabels(measures)
plt.show()


ind = np.arange(3)
width = 0.08
fig, ax = plt.subplots()
rs = []
result_dicts = defaultdict(dict)
for i, score in enumerate(scores):

    for p in params[score]:
        result_dicts[score][p] = trec_eval(score + ('_' + str(p) if p != None else ''), measures, validation=False)

    means = [result_dicts[score][p]['ndcg_cut_10']['all'] for p in result_dicts[score]]
    print(score)
    markers_on = [i for i, _ in enumerate(params[score])]
    x = np.arange(len(params[score]))
    plt.plot(x,means, marker='o')
    plt.xticks(x, params[score])
    plt.show()

In [31]:
import pandas
pairwise = defaultdict(dict)
for score in scores:
    for score2 in scores:
        pairwise[score][score2] = (100, None)
        for measure in measures:
            _, p = scipy.stats.ttest_rel(list(result_dicts[score][bests[score]][measure].values()), 
                                         list(result_dicts[score2][bests[score2]][measure].values()))[1]
        if p < pairwise[score][score2]:
            pairwise[score][score2] = (p, measure)
        # here we should compare to our p value to decide the significance
#         print('result of ', score, ' with score ', param, 'respect to ', score2, 'is ', a, p)
df = pandas.DataFrame.from_dict(pairwise)    
df = df.applymap(lambda x: x < 0.05)
df

                                             abs_discounting  \
abs_discounting                                   (nan, nan)   
bm25               (-0.5257236903056055, 0.6000503635431209)   
dirichlet_prior     (-0.820053427200254, 0.4138115557777783)   
jelinek_mercer   (3.5410519528295255, 0.0005682863686325523)   
plm-circle        (4.872483344675144, 3.404494295037945e-06)   
plm-cosine        (4.872483344675144, 3.404494295037945e-06)   
plm-gauss         (4.872483344675144, 3.404494295037945e-06)   
plm-passage       (4.872483344675144, 3.404494295037945e-06)   
plm-triangle      (4.872483344675144, 3.404494295037945e-06)   
tfidf             (-0.8849352885444004, 0.37796127448567174)   

                                                        bm25  \
abs_discounting     (0.5257236903056055, 0.6000503635431209)   
bm25                                              (nan, nan)   
dirichlet_prior   (-0.11174077747120753, 0.9112156386153231)   
jelinek_mercer   (5.652313301359162, 1.

In [23]:
for score1 in scores:
    res1 = result_dicts[score1][bests[score1]]
    # change map cut to others
    values1 = list(sorted(res1['map_cut_10'].items(), key=lambda t: t[1], reverse=True))
    for score in scores:
        res = result_dicts[score][bests[score]]
        values = list(sorted(res['map_cut_10'].items(), key=lambda t: t[1], reverse=True))
        v1 = [s for s, _ in values1[:20]]
        v = [s for s, _ in values[-40:]]
        newset = set(v1) & set(v)
        if len(newset) > 0:
            print([dictionary.id2token[int(tok)] for a in newset for tok in tokenized_queries[a]], score1, score)


['health', 'hazards', 'fine', 'diameter', 'fibers'] jelinek_mercer abs_discounting
['black', 'resistance', 'south', 'african', 'government'] abs_discounting jelinek_mercer
['black', 'resistance', 'south', 'african', 'government'] dirichlet_prior jelinek_mercer
['laser', 'research', 'applicable', 'u', 's', 's', 'strategic', 'defense', 'initiative'] bm25 jelinek_mercer
['laser', 'research', 'applicable', 'u', 's', 's', 'strategic', 'defense', 'initiative'] bm25 abs_discounting
['laser', 'research', 'applicable', 'u', 's', 's', 'strategic', 'defense', 'initiative'] bm25 dirichlet_prior


### Task 2: Latent Semantic Models (LSMs) [20 points] ###

In this task you will experiment with applying distributional semantics methods ([LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]** and [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]**) for retrieval.

You do not need to implement LSI or LDA on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. 
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

Perform analysis **[5 points]**

In [18]:
# The number of topics means the dimension of representation of each query and each document both in the Lsi and Lda case
num_topics = [10, 25, 50]

In [31]:
# import numpy as np
import gensim
from scipy import spatial

# for the LdaModel the corpus should be a list of docs, where each document is represented as a list of tuples, 
# where a tuple means a word and number of occurences in the doc
my_corpus = []

# iterate over the documents
for int_doc_id in range(index.document_base(), index.maximum_document()):
    # get the external doc ids and the tokens
    ext_doc_id, doc_token_ids = index.document(int_doc_id)
    # omit the stop words
    doc_token_ids = [doc_token_id for doc_token_id in doc_token_ids if doc_token_id > 0]
    # build the bag of words model for the document
    document_bow = collections.Counter(doc_token_ids).most_common(len(doc_token_ids))
    my_corpus.append(document_bow)

In [29]:
# Trains the lda nodel and saves it with the parameter in its name
def train_lda(num_topics):
    for num in num_topics:
        lda_model = gensim.models.LdaModel(my_corpus, id2word=id2token, minimum_probability=0, alpha='auto', num_topics=num)
        lda_model.save('Lda_model_'+str(num)+'.lda')


In [30]:
# Trains the lsi nodel and saves it with the parameter in its name
def train_lsi(num_topics):
    for num in num_topics:
        lsi_models[num] = gensim.models.LsiModel(my_corpus, id2word=id2token, num_topics=num)
        lsi_models.save('Lsi_model_'+str(num)+'.lsi')

In [32]:
# If you want to retrain the model, uncomment

# train_lsi(num_topics)

In [ ]:
# If you want to retrain the model, uncomment

# train_lda(num_topics)

In [62]:
# To load the models from file
# input parameters: number of topics, topic name
def load_model_lda(num, model_name):
    model = gensim.models.LdaModel.load(model_name+'_'+num+'.lda')
    return model
def load_model_lsi(num, model_name):
    model = gensim.models.LsiModel.load(model_name+'_'+num+'.lsi')
    return model

In [63]:
# Loading the lsi models with all the parameters
lsi_models = {}
for num in num_topics:
    lsi_models[num] = load_model_lsi(str(num), 'Lsi_model')

In [64]:
# Loading the lda models with all the parameters
lda_models = {}
for num in num_topics:
    lda_models[num] = load_model_lda(str(num), 'Lda_model')


In [65]:
# input: pretrained model (including the parameter)
# output: representation of all the documents based on the pretrained model
def create_repr_for_docs(model):
    docs_repr = []
    for int_doc_id in range(index.document_base(), index.maximum_document()):
        ext_doc_id, doc_token_ids = index.document(int_doc_id)
        doc_token_ids = [doc_token_id for doc_token_id in doc_token_ids if doc_token_id > 0]
        document_bow = collections.Counter(doc_token_ids).most_common(len(doc_token_ids))
        docs_repr.append(model[document_bow])
    return docs_repr

In [ ]:
import pandas as pd
# saving the document representation
lda_doc_repr = {}
for num in num_topics:
    lda_doc_repr[num] = create_repr_for_docs(lda_models[num])
    df = pd.DataFrame(lda_doc_repr[num])
    df.to_csv('lda_doc_repr_{}.csv'.format(num))

In [ ]:
# saving the document representation
lsi_doc_repr = {}
for num in num_topics:
    lsi_doc_repr[num] = create_repr_for_docs(lsi_models[num])
    df = pd.DataFrame(lsi_doc_repr[num])
    df.to_csv('lsi_doc_repr_{}.csv'.format(num))

In [68]:
# loading the document representations
import csv
from collections import defaultdict

def to_tuple(arr):
    res_arr = []
    tup = ()
    for item in arr:
        try:
            tup = tuple([float(i.strip()) for i in item.replace('(', '').replace(')','').split(',')])
        except Exception:
            continue
        res_arr.append(tup)
    return res_arr

lsi_doc_repr = {}
lda_doc_repr = {}
for num in num_topics:
    document_ids = {}
    with open('lsi_doc_repr_{}.csv'.format(num), 'r') as csvin:
        reader=csv.DictReader(csvin)
        data = defaultdict(list)
        for i, line in enumerate(reader):
            tuples = to_tuple(list(line.values())[1:])
            document_ids[i] = tuples
    lsi_doc_repr[num] = document_ids

    document_ids = {}
    with open('lda_doc_repr_{}.csv'.format(num), 'r') as csvin:
        reader=csv.DictReader(csvin)
        data = defaultdict(list)
        for i, line in enumerate(reader):
            tuples = to_tuple(list(line.values())[1:])
            document_ids[i] = tuples
    lda_doc_repr[num] = document_ids

In [69]:
def cos_similarity(a, b):
    # Input: dense representation (a list) of the document and query 
    # Output: cosine similarity between the vectors 
    
    cos_sim = 1 - spatial.distance.cosine(a, b)
    return cos_sim

In [ ]:
def get_cosine_sim(doc_id, query_id, model):
    query_count_list = query_term_counts[query_id].most_common(len(query_term_counts[query_id]))
    query_model = model[query_count_list]
    a = [x[1] for x in query_model]
    b = [x[1] for x in lda_docs[doc_id]]
    if len(b) < len(a):
        b = [0.00001] * len(a)
    return cos_similarity(a, b)

In [70]:
import numpy as np
lda_data = {}
lsi_data = {}

def get_data(lda_model, lda_docs):
    # loop through all the queries
    data = collections.defaultdict(list)
    for query_id in tokenized_queries:
        # convert the queries into format that is needed for lda
        query_count_list = query_term_counts[query_id].most_common(len(query_term_counts[query_id]))
        # calculate the lda representation of the query
        query_lda = lda_model[query_count_list]
        # convert it to vector format []
        a = [x[1] for x in query_lda]
        # take the 1000 most relevant docs for the query (based on tf-idf)
        most_relevant_doc_ids = top_1000[int(query_id)]
        for idx in most_relevant_doc_ids:
            b = [x[1] for x in lda_docs[idx]]
            if len(b) < len(a):
                b = [0.00001] * len(a)
            ext_doc_id, _ = index.document(int_doc_id)
            # calculate the cosine similarity of the document and query
            data[query_id].append((cos_similarity(a, b), ext_doc_id))
    return data


# writing the results
for param in lda_models:
    lda_data[param] = get_data(lda_models[param], lda_doc_repr[param])
    run_out_path = 'results/{}_{}.run'.format('lda_model', str(param))
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name='{}_{}'.format('lda_model', str(param)),
            data=lda_data[param],
            out_f=f_out,
            max_objects_per_query=1000)

    lsi_data[param] = get_data(lsi_models[param], lsi_doc_repr[param])   
    run_out_path = 'results/{}_{}.run'.format('lsi_model', str(param))
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name='{}_{}'.format('lsi_model', str(param)),
            data=lsi_data[param],
            out_f=f_out,
            max_objects_per_query=1000)

In [113]:
import matplotlib.pyplot as plt
measures = ['ndcg_cut_10', 'map_cut_1000', 'recall_1000', 'P_5']
scores = ['lda_model', 'lsi_model']

params = {'lda_model': [10, 25, 50],
         'lsi_model': [10, 25, 50]}

    
ind = np.arange(len(measures))
width = 0.08
colors = ['r', 'g', 'b', 'y', 'k', 'grey', 'purple', 'orange', 'pink']
colors = colors + colors + colors + colors
fig, ax = plt.subplots()
rs = []
#evaluation measures on with optimized hyper parameters
bests = dict()
for i, score in enumerate(scores):
    print(score, params[score])
    best, best_val, best_param, best_result, results = validation_best(score, params[score], measures,valid=False)
    bests[score] = best_param
    means = [best_result[measure]['all'] for measure in measures]
    rs.append(ax.bar(ind + width*i, means, width, color=colors[i]))
# print(bests)
ax.legend(rs, scores)
ax.set_ylabel('Scores')
ax.set_title('Scores measure')
ax.set_xticks(ind + width*10 / 2)
ax.set_xticklabels(measures)
plt.show()


lda_model [10, 25, 50]
lda_model_10
.
results/lda_model_10.run
lda_model_25
.
results/lda_model_25.run
lda_model_50
.
results/lda_model_50.run
[5, 5, 5]


TypeError: 'int' object is not subscriptable

### Task 3:  Word embeddings for ranking [10 points] ###

First create word embeddings on the corpus we provided using [word2vec](http://arxiv.org/abs/1411.2738) -- [gensim implementation](https://radimrehurek.com/gensim/models/word2vec.html). You should extract the indexed documents using pyndri and provide them to gensim for training a model (see example [here](https://github.com/nickvosk/pyndri/blob/master/examples/word2vec.py)).

Try one of the following (increasingly complex) methods for building query and document representations:
   * Average or sum the word vectors.
   * Cluster words in the document using [k-means](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) and use the centroid of the most important cluster. Experiment with different values of K for k-means.
   * Using the [bag-of-word-embeddings representation](https://ciir-publications.cs.umass.edu/pub/web/getpdf.php?id=1248).
   
Note that since we provide the implementation for training word2vec, you will be graded based on your creativity on combining word embeddings for building query and document representations.

Note: If you want to experiment with pre-trained word embeddings on a different corpus, you can use the word embeddings we provide alongside the assignment (./data/reduced_vectors_google.txt). These are the [google word2vec word embeddings](https://code.google.com/archive/p/word2vec/), reduced to only the words that appear in the document collection we use in this assignment.

In [ ]:
%%time
import copy
import gensim
import logging
import pyndri
import pyndri.compat
import sys


word2vec_init = gensim.models.Word2Vec(
    size=300,  # Embedding size
    window=5,  # One-sided window size
    sg=True,  # Skip-gram.
    min_count=5,  # Minimum word frequency.
    sample=1e-3,  # Sub-sample threshold.
    hs=False,  # Hierarchical softmax.
    negative=10,  # Number of negative examples.
    iter=1,  # Number of iterations.
    workers=8,  # Number of workers.
)

with pyndri.open('index/') as index:
    logging.info('Loading vocabulary.')
    dictionary = pyndri.extract_dictionary(index)
    sentences = pyndri.compat.IndriSentences(index, dictionary)

    logging.info('Constructing word2vec vocabulary.')

    # Build vocab.
    word2vec_init.build_vocab(sentences, trim_rule=None)
#     print(len(word2vec_init.corpus_count))
#     print(len(dictionary))
#     print(len(sentences))
    models = [word2vec_init]

    for epoch in range(1, 5 + 1):
        logging.info('Epoch %d', epoch)

        model = copy.deepcopy(models[-1])
        model.train(sentences, total_examples=len(sentences), epochs=model.iter)

        models.append(model)

    logging.info('Trained models: %s', models)

### Task 4: Learning to rank (LTR) [10 points] ###

In this task you will get an introduction into learning to rank for information retrieval, in particular pointwise learning to rank.

You will experiment with a pointwise learning to rank method, logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Train your LTR model using 10-fold cross validation on the test set.

You can explore different ways for devising features for the model. Obviously, you can use the retrieval methods you implemented in Task 1 and Task 2 as features. Think about other features you can use (e.g. query/document length). 
One idea is to also explore external sources such as Wikipedia entities (?). Creativity on devising new features and providing motivation for them will be taken into account when grading.

For every query, first create a document candidate set using the top-1000 documents using TF-IDF, and subsequently compute features given a query and a document. Note that the feature values of different retrieval methods are likely to be distributed differently.

In [ ]:
import pandas as pd
columns = ['query_id', 'session_id', 'ext_doc_id', 'rel']
train_data = pd.read_csv('ap_88_89/qrel_test', sep=' ', names = columns)[['query_id', 'ext_doc_id', 'rel']]
test_data = pd.read_csv('ap_88_89/qrel_validation', sep=' ', names = columns)[['query_id', 'ext_doc_id', 'rel']]
total_data = []
for k, v in top_1000.items():
    for doc_id in v:
        data = {}
        data['query_id'] = k
        data['ext_doc_id'] = index.ext_document_id(doc_id)
        data['rel'] = float('nan')
        total_data.append(data)
top1000_data = pd.DataFrame(total_data)
def prepare_dataset(dataset):
    all_results = []
    for idx, data in dataset.iterrows():
        print(idx)
        sample_results = {}
        query_id = str(data.query_id)
        ext_doc_id = data.ext_doc_id
        rel = data.rel
        if ext_doc_id in ext2int:
            int_doc_id = ext2int[ext_doc_id]
            sample_results['ext_doc_id'] = ext_doc_id
            sample_results['query_id'] = query_id
            sample_results['doc_len'] = document_lengths[int_doc_id]
            sample_results['query_len'] = len(tokenized_queries[query_id])
            sample_results['tfidf'] = tfidf(int_doc_id, query_id)
            sample_results['bm25'] = bm25(int_doc_id, query_id)
            sample_results['jelinek_mercer'] = jelinek_mercer(int_doc_id, query_id, lambd=bests['jelinek_mercer'])
            sample_results['dirichlet_prior'] = dirichlet_prior(int_doc_id, query_id, mu=bests['dirichlet_prior'])
            sample_results['abs_discounting'] = abs_discount(int_doc_id, query_id, delta=bests['abs_discounting'])
            sample_results['plm-gauss'] = positional_language_model(int_doc_id, query_id, 
                                                                    gauss_kernel_table, gauss_integral_table,
                                                                   mu=bests['plm-gauss'])
            sample_results['plm-triangle'] = positional_language_model(int_doc_id, query_id, 
                                                                    gauss_kernel_table, gauss_integral_table,
                                                                   mu=bests['plm-triangle'])
            sample_results['plm-circle'] = positional_language_model(int_doc_id, query_id, 
                                                                    gauss_kernel_table, gauss_integral_table,
                                                                   mu=bests['plm-circle'])
            sample_results['plm-cosine'] = positional_language_model(int_doc_id, query_id, 
                                                                    gauss_kernel_table, gauss_integral_table,
                                                                   mu=bests['plm-cosine'])
            sample_results['plm-passage'] = positional_language_model(int_doc_id, query_id, 
                                                                    gauss_kernel_table, gauss_integral_table,
                                                                   mu=bests['plm-passage'])
            for i in range(2, 11):
                sample_results['tfidf^{}'.format(i)] =  np.power(sample_results['tfidf'], i)
                sample_results['bm25^{}'.format(i)] = np.power(sample_results['bm25'], i)
                sample_results['jelinek_mercer^{}'.format(i)] = np.power(sample_results['jelinek_mercer'], i)
                sample_results['dirichlet_prior^{}'.format(i)] = np.power(sample_results['dirichlet_prior'], i)
                sample_results['abs_discounting^{}'.format(i)] = np.power(sample_results['abs_discounting'], i)
                sample_results['plm-gauss^{}'.format(i)] = np.power(sample_results['plm-gauss'], i)
                sample_results['plm-triangle^{}'.format(i)] = np.power(sample_results['plm-triangle'], i)
                sample_results['plm-circle^{}'.format(i)] = np.power(sample_results['plm-circle'], i)
                sample_results['plm-cosine^{}'.format(i)] = np.power(sample_results['plm-cosine'], i)
                sample_results['plm-passage^{}'.format(i)] = np.power(sample_results['plm-passage'], i)
            sample_results['rel'] = rel
            all_results.append(sample_results)
    return pd.DataFrame(all_results)
top1000_data = prepare_dataset(top1000_data)
top1000_data = top1000_data[list(top1000_data.columns.values)[:-1]]
train_data = prepare_dataset(train_data)
test_data =  prepare_dataset(test_data)

In [ ]:
import pickle
pickle.dump(top1000_data, open('top1000_data2.pkl', 'wb'))
pickle.dump(train_data, open('train_data.pkl', 'wb'))
pickle.dump(test_data, open('test_data.pkl', 'wb'))

In [ ]:
top1000_data.to_pickle('top1000_data.pd.pkl')
train_data.to_pickle('train_data.pd.pkl')
test_data.to_pickle('test_data.pd.pkl')

### Task 4: Write a report [20 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Only send us the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file structure as the one we have provided.
